# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели. 
Можно использовать данные из семинара или любые другие (сопоставимые или большие по объему). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг <start>  
    - еще одна матрица не нужна, можно по строкам хронить биграмы, а по колонкам униграммы  
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так). 

In [176]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML 

In [177]:
dvach = open('2ch_corpus.txt', encoding="utf8").read()
news = open('lenta.txt', encoding="utf8").read()

In [178]:
def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [179]:
norm_dvach = normalize(dvach)
norm_news = normalize(news)

In [181]:
print("Длина корпуса токсичных постов в токенах -", len(norm_dvach))
print("Длина корпуса новостных текстов в токенах - ", len(norm_news))

Длина корпуса токсичных постов в токенах - 1858941
Длина корпуса новостных текстов в токенах -  1505789


In [182]:
print("Уникальных токенов в корпусе токсичных постов -", len(set(norm_dvach)))
print("Уникальный токенов в корпусе новостных текстов - ", len(set(norm_news)))

Уникальных токенов в корпусе токсичных постов - 189515
Уникальный токенов в корпусе новостных текстов -  116302


In [183]:
from collections import Counter

In [184]:
vocab_dvach = Counter(norm_dvach)
vocab_news = Counter(norm_news)

In [185]:
probas_dvach = Counter({word:c/len(norm_dvach) for word, c in vocab_dvach.items()})
probas_dvach.most_common(20)

[('и', 0.030066580918921042),
 ('в', 0.02628001641794979),
 ('не', 0.02506911192985684),
 ('на', 0.015955320798239428),
 ('что', 0.014345802260534357),
 ('я', 0.011691602907246653),
 ('а', 0.011463516055646737),
 ('с', 0.011339789697467536),
 ('это', 0.009536074571489897),
 ('ты', 0.008321404498582796),
 ('как', 0.007882444897390503),
 ('у', 0.006848522895562581),
 ('но', 0.005786090037284669),
 ('так', 0.005383172462170666),
 ('по', 0.005060945990217011),
 ('то', 0.005049649235774562),
 ('все', 0.0046537248896011225),
 ('за', 0.004583792600195488),
 ('же', 0.004228751746289958),
 ('если', 0.004209385881531474)]

In [186]:
probas_news = Counter({word:c/len(norm_news) for word, c in vocab_news.items()})
probas_news.most_common(20)

[('в', 0.04808907489694771),
 ('и', 0.0221080111489724),
 ('на', 0.018883123731146926),
 ('по', 0.012943380513471676),
 ('что', 0.011310349590812525),
 ('с', 0.01057319451795703),
 ('не', 0.008435444806676101),
 ('из', 0.005131529052211166),
 ('о', 0.00499073907433246),
 ('как', 0.0049900749706632205),
 ('к', 0.00407161959610543),
 ('за', 0.0040125143695431435),
 ('россии', 0.0036751497055696383),
 ('для', 0.003325831175549828),
 ('его', 0.003260084912295149),
 ('он', 0.0031704309169478593),
 ('от', 0.003066830744546547),
 ('сообщает', 0.003050228152815567),
 ('а', 0.0029180715226369697),
 ('также', 0.002716184007188258)]

In [187]:
def compute_joint_proba(text, word_probas):
    prob = 0
    for word in normalize(text):
        if word in word_probas:
            prob += (np.log(word_probas[word]))
        else:
            prob += (np.log(1/len(norm_dvach)))
    
    return np.exp(prob)

In [188]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

In [189]:
compute_joint_proba(phrase, probas_dvach)

3.8958314050721132e-50

In [190]:
compute_joint_proba(phrase, probas_news)

4.573351371331133e-45

In [191]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\posch\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [192]:
from nltk.tokenize import sent_tokenize
def ngrammer(tokens, n=3):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [193]:
sentences_dvach = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] + ['<end>'] for text in sent_tokenize(dvach[:5000000])]
sentences_news = [['<start>'] + ['<start>'] + normalize(text) + ['<end>'] + ['<end>'] for text in sent_tokenize(news[:5000000])]

In [194]:
unigrams_dvach = Counter()
trigrams_dvach = Counter()

for sentence in sentences_dvach:
    unigrams_dvach.update(sentence)
    trigrams_dvach.update(ngrammer(sentence))
    
unigrams_news = Counter()
trigrams_news = Counter()

for sentence in sentences_news:
    unigrams_news.update(sentence)
    trigrams_news.update(ngrammer(sentence))

In [195]:
def compute_joint_proba_markov_assumption(text, word_counts, trigram_counts):
    prob = 0
    for ngram in ngrammer(['<start>'] + ['<start>'] + normalize(phrase) + ['<end>'] + ['<end>']):
        word1, word2, word3 = ngram.split()
        if word1 in word_counts and ngram in trigram_counts:
            prob += np.log(trigram_counts[ngram]/word_counts[word3])
        else:
            prob += np.log(2e-5)
    
    return np.exp(prob)

In [196]:
# Эта фраза более вероятна в корпусе двача
phrase = 'Безграмотное быдло с дубляжом, войсовером, порнографией и котикам'

compute_joint_proba_markov_assumption(phrase, unigrams_dvach, trigrams_dvach) > \
compute_joint_proba_markov_assumption(phrase, unigrams_news, trigrams_news)

True

In [197]:
phrase = 'Технические возможности устаревшего российского судна не позволили разгрузить его у терминала'

compute_joint_proba_markov_assumption(phrase, unigrams_dvach, trigrams_dvach) > \
compute_joint_proba_markov_assumption(phrase, unigrams_news, trigrams_news)

False

In [198]:
from scipy.sparse import lil_matrix
from numpy import array, matrix

In [206]:
# матрица слова на слова (инициализируем нулями)
matrix_dvach = lil_matrix((len(unigrams_dvach), 
                         len(unigrams_dvach)))

bigrams = array(word1, word2)
# к матрице нужно обращаться по индексам
# поэтому зафиксируем порядок слов в словаре и сделаем маппинг id-слово и слово-id
id2word_dvach = bigrams
word2id_dvach = {word : i for i, word in enumerate(bigrams)}

# заполняем матрицу
for ngram in trigrams_dvach:
    word1, word2, word3 = ngram.split()
    # на пересечение двух слов ставим вероятность встретить второе после первого
    matrix_dvach[word2id_dvach[bigrams], word2id_dvach[word3]] =  (trigrams_dvach[ngram]/unigrams_dvach[word3])


TypeError: data type '<start>' not understood

In [ ]:
# то же самое для другого корпуса
matrix_news = lil_matrix((len(unigrams_news), 
                        len(unigrams_news)))

id2word_news = list(unigrams_news)
word2id_news = {word:i for i, word in enumerate(id2word_news)}



for ngram in trigrams_news:
    word1, word2, word3 = ngram.split()
    matrix_news[word2id_news[word1, word2], word2id_news[word3]] =  (trigrams_news[ngram]/
                                                                     unigrams_news[word1])

In [207]:
def generate(matrix, id2word, word2id, n=100, start='<start>'):
    text = []
    current_idx = word2id[start]
    
    for i in range(n):
        
        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])
# просто выбирать наиболее вероятное продолжение не получится
# можете попробовать раскоментировать следующую строчку и посмотреть что получается
# в современных языковых моделях есть специальный параметр, который
# позволяет регулировать разнообразность/случайность генерации
# он называется температура, чем выше температура тем ближе будет к argmax
# чем меньше температура тем ближе к полностью рандомной генерации
#         chosen = matrix[current_idx].argmax()
        text.append(id2word[chosen])
        
        if id2word[chosen] == '<end>':
            chosen = word2id['<start>']
        current_idx = chosen
    
    return ' '.join(text)

In [208]:
print(generate(matrix_dvach, id2word_dvach, word2id_dvach).replace('<end>','\n'))

KeyError: '<start>'

In [209]:
print(generate(matrix_news, id2word_news, word2id_news).replace('<end>', '\n'))

NameError: name 'matrix_news' is not defined

Перплексия

## Задание № 2* (2 балла). 

Прочитайте главу про языковое моделирование в книге Журафски и Мартина - https://web.stanford.edu/~jurafsky/slp3/3.pdf

Развернуто (в пределах 1000 знаков) ответьте на вопросы (по-русски):

1. Что можно делать с проблемой несловарных слов? В семинаре мы просто использовали какое-то маленькое значение вероятности, а какие есть другие способы?

Если нам необходимо учесть, что слова могут не встречаться в предзаданном словаре, то мы создаём т.н. "систему открытого словаря", добавляя тег <UNK>(неизвестный). В некоторых системах любое слово, которого нет в словаре, превращается в <UNK>, и затем его вероятность считается как для любого другого слова в тренировочном сете данных. 

Другой вариант - мы не берём словарь заранее, но создаём его по мере работы. В таком случае слова заменяются на <UNK> в зависимости от частоты. Но необходимо определить размер словаря заранее, чтобы определить, насколько редким должно быть слово, чтобы мы могли заменить его на <UNK>.

Выбор модели <UNK> влияет на перплексию и подобные ей метрики. Низкая перплексия может получиться, например, если взять маленький словарь и приписать <UNK> большую вероятность. Поэтому перплексию стоит использовать для моделей с одинаковым словарём.

2. Что такое сглаживание (smoothing)?

Сглаживание необходимо, чтобы учесть слова, которые не встречаются в тренировочной выборке (но которые могут появиться в тестовой). Как правило, мы отнимаем немного вероятности у самых частотных слов и добавляем неизвестным.

Сглаживание Лапласа (сглаживание +1) - добавить единицу ко всем н-граммам перед расчётом вероятности. Не используется в современных н-грамных моделях, но для классификации текста всё ещё подходит.

Сглаживание +к - добавить к событиям, которые пока не встречались,  не единицу, а десятичную величину (0.5, 0.05, 0.01). Но для использования этого метода нужно знать, как именно мы выбираем к.

Бэк-офф и Интерполяция. Бэк-офф: если нет доказательств, что встречается триграмма, мы сужаем контекст до биграммы или даже униграммы (без доказательств не выбираем н-грамму больщого порядка). Таким образом мы "отступаем", пока не найдётся н-грамма с ненулевой вероятностью (т.е. которая уже встречалась в тренировочной выборке). С наиболее вероятных слов убирается немного вероятности, чобы у менее вероятных слов тоже был некоторый вес (т.к. они всё же могут встречаться)
Интерполяция: сочетаем оценки вероятности всех н-грамм, комбинируя триграмму, биграмму и униграмму (например, с помощью линейной интерполяции).

Сглаживание Кнезера-Нея - самое эффективное. Заключается в том, чтобы из каждого показателя вычесть фиксированное чилсло (можно ли таким образом перевести absolute discounting как "абсолютное вычитание" не уверена:)). Также при этом сглаживании учитывается контекст (если слово появлялось во множестве контекстов, тем вероятней, что оно появится в новом).

Глупый бэк-офф (глупое отступление?) - не стремится к распределению вероятности. Действует по факту: если н-грамма высокого порядка не встречается, мы отступаем к н-грамме предыдущего порядка, которой соответствует фиксированный вес вне контекста. 
